In [17]:
import curl_cffi
from curl_cffi import requests
import json
import pandas as pd
import numpy as np
import requests
import scraper_functions
from scraper_functions import VPNTester, ScraperClient
import openpyxl
import pydantic_model
from pydantic_model import categories, products
import pydantic
from pydantic import ValidationError

import random
from datetime import datetime
from zoneinfo import ZoneInfo
from pathlib import Path

##### VPN Checker

In [18]:
tester = VPNTester()
tester.vpn_check()

[🛡️  VPN Check] Your current IP: 185.239.150.15


##### Fixing impersonate agent for the session

In [19]:
FIXED_IMPERSONATE = random.choice(scraper_functions.IMPERSONATE_OPTIONS)
print(FIXED_IMPERSONATE)

chrome119


## Product Categories

##### Get json

In [20]:
categories_payload = [
  {
    "operationName": "GetCategoryTree",
    "variables": {
      "getCategoryInput": {
        "clientId": "CADENA_REAL",
        "storeReference": "2"
      }
    },
    "query": "fragment CategoryFields on CategoryModel {\n  active\n  boost\n  hasChildren\n  categoryNamesPath\n  isAvailableInHome\n  level\n  name\n  path\n  reference\n  slug\n  photoUrl\n  imageUrl\n  shortName\n  isFeatured\n  isAssociatedToCatalog\n  __typename\n}\n\nfragment CategoriesRecursive on CategoryModel {\n  subCategories {\n    ...CategoryFields\n    subCategories {\n      ...CategoryFields\n      subCategories {\n        ...CategoryFields\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment CategoryModel on CategoryModel {\n  ...CategoryFields\n  ...CategoriesRecursive\n  __typename\n}\n\nquery GetCategoryTree($getCategoryInput: GetCategoryInput!) {\n  getCategory(getCategoryInput: $getCategoryInput) {\n    ...CategoryModel\n    __typename\n  }\n}"
  },
  {
    "operationName": "getDynamicHome",
    "variables": {
      "getDynamicHomeInput": {
        "breakpointId": "medium",
        "storeReference": "2",
        "targetType": "WEB",
        "clientId": "CADENA_REAL"
      }
    },
    "query": "query getDynamicHome($getDynamicHomeInput: GetDynamicHomeInput!) {\n  getDynamicHome(getDynamicHomeInput: $getDynamicHomeInput)\n}"
  }
]

In [21]:
url = "https://nextgentheadless.instaleap.io/api/v3"

scraper = ScraperClient(
    url, 
    random_wait = True, 
    impersonate = FIXED_IMPERSONATE
)

json_response = scraper.get_json(
    method="POST", 
    payload=categories_payload
)

[⏱️] Waiting 6.80s to simulate human behavior...
[🎭 Impersonation] Using: chrome119


In [22]:
json_data = json_response[0]["data"]["getCategory"]

##### Saving raw json file

In [23]:
ScraperClient.save_json_raw(
    json_response, 
    supermarket="real", 
    subfolder=f'/workspaces/proyecto-tesis/outputs/real/categories', 
    name='categories'
)

[💾] Archivo guardado en: /workspaces/proyecto-tesis/outputs/real/categories/real_categories_2025-07-13_21-22-54.json


##### Parse JSON to Model

In [24]:
if json_response :
    categories_model = ScraperClient.parse_json_to_model(
        json_data = json_response[0]["data"]["getCategory"], 
        model_class = categories, 
        supermarket = 'real'
    )

##### Model -> Dataframe | Validation

In [25]:
df = pd.DataFrame([c.model_dump() for c in categories_model])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype                           
---  ------          --------------  -----                           
 0   id              22 non-null     object                          
 1   name            22 non-null     object                          
 2   slug            22 non-null     object                          
 3   supermarket     22 non-null     object                          
 4   ingestion_time  22 non-null     datetime64[ns, America/Asuncion]
dtypes: datetime64[ns, America/Asuncion](1), object(4)
memory usage: 1012.0+ bytes


,id,name,slug,supermarket,ingestion_time
0,Cat2,Bebidas,bebidas/Cat2,real,2025-07-13 21:22:54.982833-03:00
1,Cat1,Almacén,almacen/Cat1,real,2025-07-13 21:22:54.982833-03:00
2,Cat6,Lácteos y Quesos,lacteos-y-quesos/Cat6,real,2025-07-13 21:22:54.982833-03:00
3,Cat10,Saludables,saludables/Cat10,real,2025-07-13 21:22:54.982833-03:00
4,Cat13,Condimentos y Salsas,condimentos-y-salsas/Cat13,real,2025-07-13 21:22:54.982833-03:00


In [26]:
df.to_csv('/workspaces/proyecto-tesis/outputs/biggie/categories/biggie_categories_test.csv')

# Product Data

#### Get JSON
14 min 17 sec for full scrap (1st test on 01/07)

In [27]:
def build_payload(category_reference, page):
    return [{
        "operationName": "GetProductsByCategory",
        "variables": {
            "getProductsByCategoryInput": {
                "categoryReference": category_reference,
                "categoryId": "null",
                "clientId": "CADENA_REAL",
                "storeReference": "2",
                "currentPage": page,
                "pageSize": 100,
                "filters": {},
                "googleAnalyticsSessionId": ""
            }
        },
        "query": "fragment CategoryFields on CategoryModel {\n  active\n  boost\n  hasChildren\n  categoryNamesPath\n  isAvailableInHome\n  level\n  name\n  path\n  reference\n  slug\n  photoUrl\n  imageUrl\n  shortName\n  isFeatured\n  isAssociatedToCatalog\n  __typename\n}\n\nfragment CatalogProductTagModel on CatalogProductTagModel {\n  description\n  enabled\n  textColor\n  filter\n  tagReference\n  backgroundColor\n  name\n  __typename\n}\n\nfragment CatalogProductFormatModel on CatalogProductFormatModel {\n  format\n  equivalence\n  unitEquivalence\n  clickMultiplier\n  minQty\n  maxQty\n  __typename\n}\n\nfragment Taxes on ProductTaxModel {\n  taxId\n  taxName\n  taxType\n  taxValue\n  taxSubTotal\n  __typename\n}\n\nfragment PromotionCondition on PromotionCondition {\n  quantity\n  price\n  priceBeforeTaxes\n  taxTotal\n  taxes {\n    ...Taxes\n    __typename\n  }\n  __typename\n}\n\nfragment Promotion on Promotion {\n  type\n  isActive\n  conditions {\n    ...PromotionCondition\n    __typename\n  }\n  description\n  endDateTime\n  startDateTime\n  __typename\n}\n\nfragment PromotedModel on PromotedModel {\n  isPromoted\n  onLoadBeacon\n  onClickBeacon\n  onViewBeacon\n  onBasketChangeBeacon\n  onWishlistBeacon\n  __typename\n}\n\nfragment SpecificationModel on SpecificationModel {\n  title\n  values {\n    label\n    value\n    __typename\n  }\n  __typename\n}\n\nfragment NutritionalDetailsInformation on NutritionalDetailsInformation {\n  servingName\n  servingSize\n  servingUnit\n  servingsPerPortion\n  nutritionalTable {\n    nutrientName\n    quantity\n    unit\n    quantityPerPortion\n    dailyValue\n    __typename\n  }\n  bottomInfo\n  __typename\n}\n\nfragment Promotions on PromotionV2 {\n  type\n  description\n  promotionReference\n  startDateTime\n  endDateTime\n  isActive\n  conditions {\n    field\n    operator\n    values\n    value\n    __typename\n  }\n  benefit {\n    type\n    label\n    value\n    values\n    imagesURL\n    __typename\n  }\n  __typename\n}\n\nfragment CatalogProductModel on CatalogProductModel {\n  name\n  price\n  photosUrl\n  unit\n  subUnit\n  subQty\n  description\n  sku\n  ean\n  maxQty\n  minQty\n  clickMultiplier\n  nutritionalDetails\n  isActive\n  slug\n  brand\n  stock\n  securityStock\n  boost\n  isAvailable\n  location\n  priceBeforeTaxes\n  taxTotal\n  promotion {\n    ...Promotion\n    __typename\n  }\n  taxes {\n    ...Taxes\n    __typename\n  }\n  categories {\n    ...CategoryFields\n    __typename\n  }\n  categoriesData {\n    ...CategoryFields\n    __typename\n  }\n  formats {\n    ...CatalogProductFormatModel\n    __typename\n  }\n  tags {\n    ...CatalogProductTagModel\n    __typename\n  }\n  specifications {\n    ...SpecificationModel\n    __typename\n  }\n  promoted {\n    ...PromotedModel\n    __typename\n  }\n  score\n  relatedProducts\n  ingredients\n  stockWarning\n  nutritionalDetailsInformation {\n    ...NutritionalDetailsInformation\n    __typename\n  }\n  productVariants\n  isVariant\n  isDominant\n  promotions {\n    ...Promotions\n    __typename\n  }\n  seals\n  previousPrice\n  previousPricePerSubUnit\n  pricePerSubUnit\n  hasAgeRestriction\n  __typename\n}\n\nfragment CategoryWithProductsModel on CategoryWithProductsModel {\n  name\n  reference\n  level\n  path\n  hasChildren\n  active\n  boost\n  isAvailableInHome\n  slug\n  photoUrl\n  categoryNamesPath\n  imageUrl\n  shortName\n  isFeatured\n  products {\n    ...CatalogProductModel\n    __typename\n  }\n  __typename\n}\n\nfragment PaginationTotalModel on PaginationTotalModel {\n  value\n  relation\n  __typename\n}\n\nfragment PaginationModel on PaginationModel {\n  page\n  pages\n  total {\n    ...PaginationTotalModel\n    __typename\n  }\n  __typename\n}\n\nfragment AggregateBucketModel on AggregateBucketModel {\n  min\n  max\n  key\n  docCount\n  __typename\n}\n\nfragment AggregateModel on AggregateModel {\n  name\n  docCount\n  buckets {\n    ...AggregateBucketModel\n    __typename\n  }\n  __typename\n}\n\nfragment BannerModel on BannerModel {\n  id\n  storeId\n  title\n  desktopImage\n  mobileImage\n  targetUrl\n  targetUrlInfo {\n    type\n    url\n    __typename\n  }\n  targetCategory\n  index\n  categoryId\n  __typename\n}\n\nfragment CarouselModel on CarouselModel {\n  id\n  name\n  autoplaySpeed\n  lazyLoading\n  isActive\n  createdAt\n  updatedAt\n  banners {\n    id\n    name\n    webImageUrl\n    tabletImageUrl\n    appImageUrl\n    redirectUrl\n    redirectMode\n    isActive\n    __typename\n  }\n  position\n  __typename\n}\n\nquery GetProductsByCategory($getProductsByCategoryInput: GetProductsByCategoryInput!) {\n  getProductsByCategory(getProductsByCategoryInput: $getProductsByCategoryInput) {\n    category {\n      ...CategoryWithProductsModel\n      __typename\n    }\n    pagination {\n      ...PaginationModel\n      __typename\n    }\n    aggregates {\n      ...AggregateModel\n      __typename\n    }\n    carousels {\n      ...CarouselModel\n      __typename\n    }\n    banners {\n      ...BannerModel\n      __typename\n    }\n    promoted {\n      ...PromotedModel\n      __typename\n    }\n    __typename\n  }\n}"
    }
]

In [28]:
# Original sorting
categories = df["id"].unique() #[18:-2] subset for testing
print(categories.size)
categories

22


array(['Cat2', 'Cat1', 'Cat6', 'Cat10', 'Cat13', 'Cat3', 'Cat4', 'Cat12',
       'Cat5', 'Cat9', 'Cat8', 'Cat15', 'Cat11', 'Cat16', 'Cat7', 'Cat22',
       'Cat20', 'Cat14', 'Cat18', 'Cat21', 'Cat23', 'Cat25'], dtype=object)

In [29]:
# Shuffleled
categories = df["id"].unique() #[18:-2] subset for testing
np.random.shuffle(categories)
print(categories.size)
categories

22


array(['Cat23', 'Cat2', 'Cat9', 'Cat6', 'Cat7', 'Cat4', 'Cat8', 'Cat5',
       'Cat21', 'Cat25', 'Cat11', 'Cat16', 'Cat22', 'Cat12', 'Cat14',
       'Cat1', 'Cat13', 'Cat18', 'Cat15', 'Cat3', 'Cat10', 'Cat20'],
      dtype=object)

In [30]:
scraper = ScraperClient(
    url="https://nextgentheadless.instaleap.io/api/v3",
    random_wait=True,
    impersonate=FIXED_IMPERSONATE
)

In [31]:
json_response = []

for category in categories:
    page = 1
    while True:
        payload = build_payload(category_reference=category, page=page)
        data = scraper.get_json(method="POST", payload=payload)

        if not data:
            print(f"[⚠️] No se obtuvo respuesta en categoría {category}, página {page}")
            break

        # Extraer los productos desde el JSON anidado
        products = (
            data[0].get("data", {})
                .get("getProductsByCategory", {})
                .get("category", {})
                .get("products", [])
        )

        # Si no hay productos, salimos del while
        if not products:
            print(f"[ℹ️] No hay más productos en {category} página {page}")
            break

        print(f"[📦] Scrapeando {category}, página {page} ({len(products)} productos)")
        json_response.extend(products)

        page += 1

[⏱️] Waiting 7.07s to simulate human behavior...


[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat23, página 1 (40 productos)
[⏱️] Waiting 9.81s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[ℹ️] No hay más productos en Cat23 página 2
[⏱️] Waiting 7.25s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat2, página 1 (100 productos)
[⏱️] Waiting 9.45s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat2, página 2 (100 productos)
[⏱️] Waiting 3.04s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat2, página 3 (100 productos)
[⏱️] Waiting 2.88s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat2, página 4 (100 productos)
[⏱️] Waiting 3.61s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat2, página 5 (100 productos)
[⏱️] Waiting 8.64s to simulate human behavior...
[🎭 Impersonation] Using: chrome119
[📦] Scrapeando Cat2, página 6 (100 productos)
[⏱️] Waitin

##### Saving raw json file

In [32]:
ScraperClient.save_json_raw(
    json_response, 
    supermarket="real", 
    subfolder=f'/workspaces/proyecto-tesis/outputs/real/products', 
    name='products'
)

[💾] Archivo guardado en: /workspaces/proyecto-tesis/outputs/real/products/real_products_2025-07-13_21-41-03.json


##### Parse JSON to Model

##### Model -> Dataframe | Validation

json_data = json_response[0]["data"]["getProductsByCategory"]["category"]["products"]

df = pd.DataFrame(json_data)
df.info()
df.head()